In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sbs
import matplotlib.ticker as mtick

In [ ]:
# Script parameters
input_csv = "../results/parsec-phoenix.csv"
baseline = "x86_64,qemu,qemu"

In [ ]:
# Parsing baseline argument
base_arch, base_runtime, base_tag = baseline.split(',')

In [ ]:
# Read input file
df = pd.read_csv(input_csv, sep=';')
df

In [ ]:
# Extract baseline from dataframe
base_df = df.loc[(df['arch'] == base_arch) & (df['runtime'] == base_runtime) & (df['tag'] == base_tag)]
base_df

In [ ]:
# Get the mean for each baseline benchmark
base_means = {}
for b in set(base_df['bench']):
    base_means[b] = np.array(base_df.loc[base_df['bench'] == b]['value'].values, dtype=np.float32).mean()

In [ ]:
# Print the mean of every benchmark for each runtime
mean_df = pd.DataFrame()
for b in sorted(set(df['bench'])):
    df_b = df.loc[df['bench'] == b]
    tmp_dict = { 'bench': b }
    for t in set(df_b['tag']):
        df_b_t = df_b.loc[df_b['tag'] == t]
        tmp_dict[t] = np.mean(df_b_t['value'])
    mean_df = mean_df.append(tmp_dict, ignore_index=True)
mean_df

In [ ]:
# Normalize all results from original df to these means
df_norm = pd.DataFrame(columns=['arch', 'bench', 'dataset', 'threads', 'unit', 'value', 'runtime',
                                'tag', 'norm', 'label'])
norm_vals = []
for row in df.itertuples():
    try:
        if row.arch == base_arch and row.runtime == base_runtime and row.tag == base_tag:
            continue
        # norm = base_means[row.bench] / float(row.value)      # speedup
        norm = 100 * float(row.value) / base_means[row.bench]    # relative perf
        
        # norm = 100 * (base_means[row.bench] - float(row.value)) / base_means[row.bench]
        dct = row._asdict()
        dct['norm'] = norm
        dct['label'] = f"{dct['tag']}"
        # dct['label'] = f"{dct['runtime']}-{dct['tag']}"
        del dct['Index']
        del dct['cmdline']
        norm_vals.append(dct)
    except KeyError:
        pass
df_norm = df_norm.append(norm_vals, ignore_index=True)
df_norm

In [ ]:
# refactor xticks to remove benchmark suite prefix
xlabels = []
xticks = []
for idx, b in enumerate(sorted(set(df_norm['bench']))):
    if b.startswith("parsec."):
        xlabels.append(b[7:])
        xticks.append(idx)
    else:
        xlabels.append(b[8:])
        xticks.append(idx)

# Plot
fig = plt.figure(figsize=(10, 3), dpi=500)
sbs.set(style="whitegrid")
palette = {
    'orange': '#faa200',
    'sky blue': '#00b7ec',
    'bluish green': '#00a077',
    'yellow': '#f5e636',
    'blue': '#0077b8',
    'vermillion': '#f3640d',
    'reddish purple': '#e47ead'
}
ax = sbs.barplot(data=df_norm, ci='sd',
                 x='bench', y='norm',
                 hue='label',# palette=palette,
                 order=sorted(set(df_norm['bench'])), hue_order=['no-fences', 'tcg-tso', 'risotto', 'native'])
plt.grid(visible=True, axis='y')
plt.xticks(ticks=xticks, labels=xlabels, rotation=30, ha="right", fontsize='xx-small')
ax.set_axisbelow(True)
plt.xlabel("")
max_val = max(df_norm['norm'].values)
plt.ylim(0, max_val*1.05)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel("Run time w.r.t. QEMU")
plt.axhline(y=100, xmin=0, xmax=1, color='tomato', linewidth=2.5)
# Annotate the raw value of the baseline
for idx, value in enumerate(sorted(set(base_means))):
    tmp = df_norm.loc[df_norm['bench'] == value]['norm'].values.mean()
    v = base_means[value]
    if v < 10:
        v_str = f"{v:.1f}"
    else:
        v_str = f"{v:.0f}"
    plt.text(idx, max(tmp, max_val), v_str, fontsize='xx-small', color='tomato', ha='center')
    
# Set color + hatch
style = {
    'fill': [ True, True, True, True ],
    'color': [ palette['vermillion'], palette['sky blue'], palette['bluish green'], palette['orange'] ],
    'hatch': [ '', '///', '', ''],
    'label': ['no-fences [incorrect]', 'tcg-ver', 'risotto', 'native'],
    'edgecolor': [ 'black', 'black', 'black', 'black' ]
}
for idx, bar in enumerate(ax.patches):
    bar_nr = int(idx / len(base_means))
    bar.set(color=style['color'][bar_nr], fill=style['fill'][bar_nr],
            hatch=style['hatch'][bar_nr], edgecolor=style['edgecolor'][bar_nr])
    
# parsec / phoenix separation
#plt.vlines([ 8.5 ], ymin=0, ymax=1.2, linestyle='solid', colors='black', linewidth=2.5, zorder=10)
#matplotlib.text(.4, .1, "parsec", xycoords='axes points')

handles, labels = ax.get_legend_handles_labels()
plt.legend(labels=style['label'], handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.15),
           borderaxespad=0, ncol=4, fontsize='x-small')

In [ ]:
fig.savefig("fig12.pdf", dpi=500, bbox_inches='tight')

In [ ]:
# Compute fence cost
fence_cost = 100 * (1 - mean_df['no-fences'] / mean_df['qemu'])
print(f"Average time spent on fences: {fence_cost.mean():.2f}%")
fence_cost

In [ ]:
# tcg-tso gain
for b in sorted(set(df_norm['bench'])):
    d = df_norm.loc[df_norm['bench'] == b]
    d = d.loc[d['tag'] == 'tcg-tso']
    print(f"{b}: {d['value'].values.mean():.2f}")

In [ ]:
mean_df['tcg-tso-pct'] = 100 * (1 - mean_df['tcg-tso'] / mean_df['qemu'])
mean_df

In [ ]:
mean_df['tcg-tso-pct'].values.mean()